In [1]:
import pandas as pd
import numpy as np
import torch
from torchvision import datasets,transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os
import random
import cv2
import numpy as np
from PIL import Image


In [2]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

In [3]:
train.describe()

In [5]:
train.head()

In [6]:
idcol = train['Id']
idcol.head()

In [7]:
direct = '../input/petfinder-pawpularity-score/train'
for i in range(idcol.shape[0]):
    idcol[i] = os.path.join(direct, f'{idcol[i]}.jpg')


In [8]:
train.head()

In [9]:
edtrain = train.drop(['Id'],axis = 1)

In [10]:
corr_matrix = edtrain.corr()
sns.heatmap(corr_matrix)

In [11]:
sns.jointplot(x="Info", y="Collage", data=edtrain, kind="scatter")
sns.jointplot(x="Face", y="Eyes", data=edtrain, kind="scatter")

In [12]:
fig, ax = plt.subplots(1,1, figsize=(12, 9))
sns.histplot(x='Pawpularity', data=train)
plt.show()

In [13]:
class Pawpularity(Dataset):
    def __init__(self,path,targets,transform = None):
        self.path = path
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self,idx):
        ipath = self.path[idx]
        image = Image.open(ipath)
        #image = cv2.imread(ipath)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        conv = transforms.ToTensor()
        image = conv(image)
        #image = torch.rot90(image,1,[1,2])
        if self.transform is not None:
            image = self.transform(image)
        #print(image.size)
        #conv = transforms.ToTensor()
        #image = conv(image)
        image = torch.rot90(image,1,[1,2])
        
        label = torch.tensor(self.targets[idx]).float()
        return image,label

In [14]:
transformations = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(192),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [15]:
paths = train['Id']
target = train['Pawpularity']
cutedata = Pawpularity(path  = paths.values,targets = target , transform = transformations)


In [16]:
def imageplot(dataset = cutedata, count =3):
    plt.figure(figsize = (20,20))
    for i in range(count):
        rand = random.randint(0, len(dataset))
        image,label = dataset[rand]
        plt.subplot(1, count, i%count +1)
        plt.axis('off')
        plt.imshow(image.permute(2, 1, 0))
        plt.title(f'Pawpularity: {label}')

In [17]:
for i in range(4):
    imageplot(count=4)

In [18]:
train_loader, val_loader = torch.utils.data.random_split(cutedata,[9912-1982,1982],generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(dataset=train_loader, 
                          batch_size=64, 
                          shuffle=True)

val_loader = DataLoader(dataset=val_loader, 
                         batch_size=64, 
                         shuffle=False)

In [19]:
def flatten(x):
    N = x.shape[0]
    return x.view(N, -1) 

In [20]:
class skipper(nn.Module):
    def __init__(self,width):
        super(skipper,self).__init__()
        #making a res block using sequential inside module hehe
        self.resb1 = nn.Sequential(
        nn.Conv2d(in_channels = width[0] , 
                  out_channels = width[1], 
                  kernel_size= (3,3),
                  stride = (1,1),
                  padding = 'same',
                  bias = True
                 ),
        nn.BatchNorm2d(width[1]),
        nn.ReLU(),
        nn.Conv2d(in_channels = width[1],
                  out_channels = width[2],
                  kernel_size = (3,3),
                  stride = (1,1),
                  padding = 'same',
                  bias = True
                 ),
        nn.BatchNorm2d(width[2]),
        nn.MaxPool2d(kernel_size = (2,2),stride = (2,2),padding = 0)
                  
        )
        self.skip = nn.Sequential(
        nn.Conv2d(in_channels = width[0],
                  out_channels = width[2],
                  kernel_size = (1,1),
                  padding = 'same',
                  bias = True
                 ),
        nn.BatchNorm2d(width[2]),
        nn.MaxPool2d(kernel_size = (2,2),stride = (2,2),padding = 0)
        )
    def forward(self,x):
        #cut = x
        out1 = self.resb1(x)
        out2 = self.skip(x)
        x = F.relu(out1+out2)
        return x
        
        

In [21]:
class network(nn.Module):
    def __init__(self):
        super(network,self).__init__()
        self.block1 =  skipper(width= [3,6,9])
        self.reduce = nn.MaxPool2d(kernel_size = (2,2),stride = (2,2),padding = 0)
        self.block2 =  skipper(width= [9,12,15])
        self.fc1    =  nn.Linear(15*12*12,200)
        self.fc2    =  nn.Linear(200,1)
        
    def forward(self,x):
        out = self.block1(x)
        out = self.reduce(out)
        out = self.block2(out)
        out = self.reduce(out)
        out = flatten(out)
        out = self.fc1(out)
        score = self.fc2(out)
        return score
        
 

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if(torch.cuda.is_available()):
  print("The device is " + torch.cuda.get_device_name())
else:
    print("Using cpu no gpu available")
print('\n',device)

In [23]:
#hyperparameterspace
lr = 1e-2
regp = 1e-3
epochs = 15
batch_size = 64

In [24]:
first_model = network()
first_model = first_model.to(device)
optimizer = torch.optim.Adam(first_model.parameters(),lr = lr)

In [25]:
def metric(model,data):
    msesum = 0
    N = len(data)
    for i, (image, target) in enumerate(data):
        print(i)
        image = image.to(device)
        target = target.to(device)
        scores = model(image)
        #print(scores.shape)
        a,b = scores.shape
        target = torch.reshape(target,(a,1))
        function = nn.MSELoss()
        mse   = function(scores,target)
        msesum +=mse
    return np.sqrt(msesum/N)

In [26]:
a = metric(first_model,train_loader)    

In [ ]:
for epoch in range(epochs):
    first_model = first_model.train()
    for bno,(image,target) in enumerate(train_loader):
        image = image.to(device)
        target = target.to(device)
        
        scores = first_model(image)
        a,b = scores.shape
        target = torch.reshape(target,(a,1))
        #print(scores.shape,target.shape)
        function = nn.MSELoss()
        loss = function(scores,target)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        if not bno % 15:
            print ('Epoch: %03d/%03d | Batch %06d/%06d | Cost: %.4f' 
                   %(epoch+1, epochs, bno, 
                     len(train_loader)//batch_size, loss))
            
    first_model = first_model.eval() 
    with torch.set_grad_enabled(False): 
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, epochs, 
              metric(first_model, train_loader)))
        if(epoch ==epochs-1):
             print('Epoch: %03d/%03d val accuracy: %.2f%%' % (
               epoch+1, epochs, 
               metric(first_model, val_loader)))
        
        
        